In [35]:
def a_method(a, b)
  a + yield(a, b)
end

a_method(1, 2) {|x, y| (x + y) * 3}

10

只有在调用一个方法时，才可以定义一个块。块会被直接传递给这次方法，该方法可以用`yield`关键字调用这个块。

块可以拥有自己的参数，比如上面例子中的`x`和`y`。当回调块时，你可以像调用方法那样为块提供参数。另外，像方法一样，块的最后一行代码执行的结果会被作为返回值。

在一个方法里，你可以询问当前的方法调用是后包含块。这可以通过`Kernel#block_given?`方法做到

In [36]:
def a_method
  return yield if block_given?
  'no block'
end

p a_method
p a_method {"here's a block!"}

"no block"
"here's a block!"


"here's a block!"

In [37]:
module Kernel
  def with(resource)
    begin
      yield
    ensure
      resource.dispose
    end
  end
end

:with

定义一个块的时候，它会获取环境中的绑定。当块被传给一个方法时，它会带着这些绑定给一块进入方法

In [38]:
def my_method
  x = "Goodbye"
  yield("cruel")
end

x = "hello"
my_method {|y| "#{x}, #{y} wolrd"}

"hello, cruel wolrd"

创建代码块时候，你会获得局部绑定（比如上面的`x`），然后把代码块连同它的绑定传给一个方法。在上面的例子中，代码块的绑定中包括一个名为`x`的变量。虽然在方法中也定义了一个变量`x`，但代码块看到的`x`还是在代码块定义时绑定的`x`，方法中的`x`对于这个代码块来说是不可以见的。

还可以在代码块内定义额外的绑定，但这些绑定在代码块结束时就消失了。

In [39]:
def just_yield
  yield
end

top_level_variable = 1

just_yield do
  top_level_variable += 1
  local_to_block = 1
end

p top_level_variable
local_to_block

2


NameError: undefined local variable or method `local_to_block' for main:Object

基于这样的特性，人们喜欢把代码块称为闭包(`closure`)。换句话说，代码块可以获取局部绑定，并一直带着他们。

下面的例子演示了程序运行时作用域是如何切换的，`Kernel#local_variable`方法用来跟踪绑定的名字：

In [40]:
v1 = 1
class MyClass
  v2 = 2
  p local_variables
  def my_method
    v3 = 3
    p local_variables
  end
  p local_variables
end

obj = MyClass.new
obj.my_method
obj.my_method
local_variables

[:v2]
[:v2]
[:v3]
[:v3]


[:_i29, :_29, :_i28, :_28, :_i27, :_27, :_i26, :_26, :_i25, :_25, :_i24, :_24, :_i23, :_23, :myclass, :_i22, :_22, :_i21, :_21, :_i20, :_20, :_i19, :_19, :_i18, :_18, :_i17, :_17, :_i16, :_16, :_i15, :_15, :_i14, :_14, :obj, :_i13, :_13, :_i12, :_12, :_i11, :_11, :v1, :top_level_variable, :_i10, :_10, :_i9, :_9, :x, :_i8, :_8, :_i7, :_7, :_i6, :_6, :_i5, :_5, :_i4, :_4, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :___, :_i1, :_1, :__, :_, :_dir_, :_file_, :_ex_, :_pry_, :_out_, :_in_, :_oh, :_ih, :title]

在某些语言(如`Java`和`C#`中)，有“内部作用域(inner scope)”的概念。在内部作用域里可以看到”外部作用域”的变量。但`ruby`没有嵌套式的作用域，它的作用域是截然分开的：一旦进入一个新的作用域，原先的绑定会被被替换为一组新的绑定。这意味着在程序进入`MyClass`之后，`v1`就落在了作用域之外，从而不可见了。

在i定义`MyClass`的作用域中，程序定义了`v2`以及一个方法。因为方法中的代码还没有被执行，所以直到类定义结束前，程序不会再打开一个新的作用域。在方法定义完成之后，用`class`关键字打开的作用域会永远关闭，同时程序回到顶级作用域。

全局变量可以在 **任何作用域**访问

In [41]:
def a_scope
  $var = "some value"
end

def another_scope
  $var
end


:another_scope

In [42]:
a_scope
another_scope

"some value"

谁都可以设置全局变量，你几乎没办法确定是谁修改了他们。因此，能不使用全局变量，就i尽量不要使用。有时可以使用顶级实例变量代替全局变量。它们是顶级对象`main`的实例变量。

In [43]:
@var  = "The top level @var"

def my_method
  @var
end

my_method

"The top level @var"

In [48]:
class MyClass2
  def my_method
    @var = "This is not the top-level @var!"
    $var
    p @var
    p $var
  end
end

myclass = MyClass2.new
myclass.my_method

"This is not the top-level @var!"
"some value"


"some value"

准备地说，程序会在三个地方关闭钱一个作用域，同事打开一个新的作用域

- 类定义
- 模块定义
- 方法